# Temporary Files and Directories

- Automation scripts often need scratch space for intermediate data without cluttering the filesystem or risking name collisions.  
- Hardcoding names like `/tmp/my_file.txt` can lead to security issues, collisions, and manual cleanup.  
- The `tempfile` module provides secure, unique temporary files and directories with optional automatic cleanup. 

## Why Use the tempfile Module?

- It creates files with secure default permissions, preventing unauthorized access on multiuser systems.  
- It generates unique names automatically, avoiding collisions when multiple script instances run concurrently.  
- It integrates with context managers (`with`), enabling automatic cleanup of resources when they're no longer needed.  
- It works across Windows, macOS, and Linux, choosing an appropriate temp location on each platform.  

In [3]:
import tempfile
import os

temp_dir = tempfile.gettempdir()
print(f"Default temp directory: {temp_dir}")
print(f"Sample contets: {os.listdir(temp_dir)}")

Default temp directory: /tmp
Sample contets: ['minikube_stop_c596e041b11c5da744ea537d8e9685642c24fc58_0.log', 'juju-mk8d1c95e47c9c33ab9749ecc6fa9d11a074b4dc', 'systemd-private-ab64e70ea4b54c4ab99e75e43644879f-systemd-logind.service-ndVIKs', 'systemd-private-ab64e70ea4b54c4ab99e75e43644879f-systemd-resolved.service-dzkCFQ', 'snap-private-tmp', 'systemd-private-ab64e70ea4b54c4ab99e75e43644879f-systemd-timesyncd.service-Z2e4YR', '.X11-unix', 'systemd-private-ab64e70ea4b54c4ab99e75e43644879f-wsl-pro.service-WIZJyE', 'juju-mkad63362cf163f3dcd987a3e6840a2122c94110', 'juju-mk0c05d4d080b7cdfc65fc61ffc33c56f44f92e6', 'pyright-4388-qJxgKGW1NzRx', 'pyright-5440-xFVc7KTDhuR5', 'juju-mkcb50bbf2497b2c6db0f67efca36f41e3758d4e', 'hoos-code-zsh', 'python-languageserver-cancellation', 'minikube_status_79e61c7abd88c28fb95176d20bf4f2df14921bf5_0.log', 'minikube_service_8359900fc898385f5a2e00d238b9ef086a1c2e9e_0.log', 'juju-mk896fa22ad8a057cab667b13e655439285b9c34', 'juju-mkd2397998205c82a8eed3842c428bf5a2

## tempfile.TemporaryFile()

- Creates an unnamed temporary file opened in binary or text mode.  
- On UNIX-like systems it typically has no name in the filesystem; on Windows it may appear but remains temporary.  
- The file is deleted automatically when closed or when the context block exits.  
- Ideal for internal scratch space that doesn’t need to be passed to external processes.  

In [6]:
import tempfile

with tempfile.TemporaryFile(mode="w+t", encoding="utf-8") as temp_file:
    print(type(temp_file))
    temp_file.write("This is some temporary data")
    temp_file.seek(0)
    print("Content from TemporaryFile:")
    print(temp_file.read())

<class '_io.TextIOWrapper'>
Content from TemporaryFile:
This is some temporary data


## tempfile.NamedTemporaryFile()

- Creates a temporary file with a visible name in the filesystem.  
- Default `delete=True` removes the file when closed; `delete=False` leaves it for manual cleanup.  
- Use when you need to pass a filename to another process or library.  
- Supports custom `suffix`, `prefix`, and `dir` parameters for naming and placement.  

In [8]:
import tempfile
from pathlib import Path

with tempfile.NamedTemporaryFile(mode="w+t", encoding="utf-8", suffix=".log") as temp_file:
    path = Path(temp_file.name)
    print(f"Created temp file at {path}. Exists: {path.exists()}")

Created temp file at /tmp/tmpf2i0tcs8.log. Exists: True


## tempfile.TemporaryDirectory()

- Creates a new temporary directory, returned as a path string.  
- When used in a `with` block, the directory and everything inside it are deleted on exit.  
- Ideal for workflows that produce multiple temporary files or subdirectories.  

In [13]:
import tempfile
from pathlib import Path
import time

temp_path = None

with tempfile.TemporaryDirectory(prefix="batch_job") as temp_dir:
    print(f"{temp_dir} - type: {type(temp_dir)}") 
    temp_path = Path(temp_dir)
    (temp_path/"file1.txt").write_text("Data")
    subdir = temp_path / "subdir"
    subdir.mkdir(exist_ok=True)
    (subdir/"file2.txt").write_text("Data2")
    print(f"Contests: {[p.name for p in temp_path.iterdir()]}")


print(f"After close. Exists: {temp_path.exists()}")

/tmp/batch_job0vaj8f_c - type: <class 'str'>
Contests: ['subdir', 'file1.txt']
After close. Exists: False


## Common Pitfalls & How to Avoid Them

- Calling `os.rmdir()` or `Path.rmdir()` on a non-empty directory raises an error; use `shutil.rmtree()` for recursive deletion.  
- Forgetting to delete files created with `delete=False` in `NamedTemporaryFile` can leave orphaned files.  
- On Windows, other processes can’t open an open temporary file. Use `delete=False` and close it before sharing the name.  
- Relying on a temporary file’s name after closing a `TemporaryFile` is impossible, since it may never have had one.  